In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import json

In [ ]:
tb_json = json.dumps(tax_brackets)

In [42]:
with open('Data/incomeTax.json') as data_file:    
    data = json.load(data_file)

print data[0]["data"]["New Brunswick"][0]["rate"]
print data[0]["data"]["New Brunswick"][0]["upper"]

9.68
39305


In [65]:
def getAvgRate(year, govt, income, data):
    # Fetch the federal tax bracket data
    fed_brackets = data["Federal"]
    
    untaxed_income = income # This will keep track of the part of the income that has not been taxed yet
    fed_tax_paid = 0.0 # A running total of the amount of federal tax paid
    lower = 0 # The lower value of tax bracket
    n = 0 # Keeps track of which tax bracket we're in
    while untaxed_income > 0:
        # Fetch the current bracket
        bracket = fed_brackets[n]
        # This checks to see if we're on the final bracket
        try:
            upper = int(bracket["upper"]) # Upper value of tax bracket
            rate = float(bracket["rate"])/100.0 # Tax rate of bracket
        except:
            upper = "max" # There is no upper value for the final bracket
            rate = float(bracket["rate"])/100.0

        if upper == "max" or income < upper:
            # Does the calculation for the final bracket the person falls into
            fed_tax_paid = fed_tax_paid + untaxed_income*rate
            untaxed_income = 0
        else:
            # Gets the tax paid on the brackets a person is totally above
            fed_tax_paid = fed_tax_paid + (upper - lower)*rate
            untaxed_income = untaxed_income - (upper - lower)
            lower = upper
            n = n + 1
    # This is the average rate the person ends up paying
    avg_fed_tax = round(fed_tax_paid / income * 100.0, 2)
    
    # Fetch the provincial tax brackets
    prov_brackets = data[govt]
    # Reset the amount for the provincial portion
    untaxed_income = income
    
    prov_tax_paid = 0.0
    lower = 0
    n = 0
    while untaxed_income > 0:
        bracket = prov_brackets[n]
        try:
            upper = int(bracket["upper"])
            rate = float(bracket["rate"])/100.0
        except:
            upper = "max"
            rate = float(bracket["rate"])/100.0

        if upper == "max" or income < upper:
            prov_tax_paid = prov_tax_paid + untaxed_income*rate
            untaxed_income = 0
        else:
            # Gets the tax paid on the brackets a person is totally above
            prov_tax_paid = prov_tax_paid + (upper - lower)*rate
            untaxed_income = untaxed_income - (upper - lower)
            lower = upper
            n = n + 1
    # This is the average rate the person ends up paying
    avg_prov_tax = round(prov_tax_paid / income * 100.0, 2)
    
    avg_tax_total = round(avg_prov_tax + avg_fed_tax, 2)

    return {"avg_fed": avg_fed_tax, "avg_prov": avg_prov_tax, "avg_total": avg_tax_total}

In [70]:
year = data[0]["year"]
year_data = data[0]["data"]
tax = getAvgRate(year, "Nova Scotia", 10000, year_data)
print tax

{'avg_total': 23.79, 'avg_prov': 8.79, 'avg_fed': 15.0}


In [ ]:
def allBracketsForYear(year, province, incomeList, brackets):
    incomeTotals = collections.OrderedDict()
    for income in incomeList:
        taxes = findBracket(year, province, income, brackets)
        incomeTotals[income] = taxes["total"]
    return incomeTotals
totals = allBracketsForYear("2000", "NB", incomes, tax_brackets)

In [ ]:
total_tax_year_provt = collections.OrderedDict()
for year in tax_brackets:
    for provt in tax_brackets[year]:
        if provt != "FED":
            # total_tax_year_provt[year][provt] = 